In [1]:
'''
In this checkpoint, we are going to work on the 'Credit Card Dataset for Clustering' dataset provided by Kaggle.
Dataset description : This dataset was derived and simplified for learning purposes. 
It includes usage behaviour of about 9000 active credit card holders during 6 months period.
This case requires to develop a customer segmentation to define marketing strategy.'''

"\nIn this checkpoint, we are going to work on the 'Credit Card Dataset for Clustering' dataset provided by Kaggle.\nDataset description : This dataset was derived and simplified for learning purposes. \nIt includes usage behaviour of about 9000 active credit card holders during 6 months period.\nThis case requires to develop a customer segmentation to define marketing strategy."

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('Credit_card_dataset.csv')
data.sample(3)

FileNotFoundError: [Errno 2] No such file or directory: 'Credit_card_dataset.csv'

# UNDERSTANDING MY DATASET 

In [ ]:
data.head(3)

In [ ]:
data.tail(3)

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.columns

# EDA

In [ ]:
data.columns

In [ ]:
# UNIVARIETE ANALYSIS
plt.hist(data['CREDIT_LIMIT'],bins=50)
plt.xlabel("CREDIT_LIMIT FREQUENCY")
plt.ylabel("FREQUENCY")

In [ ]:
# BIVARIATE ANALYSIS
plt.scatter(data['CREDIT_LIMIT'][:50],data.PURCHASES[:50],marker="o",color="green",label = "purchases")
plt.xlabel("PURCHASES")
plt.ylabel("CREDIT_LIMIT")
plt.legend()

In [ ]:
# plt.scatter(data['CREDIT_LIMIT'][:50],data.PURCHASES[:50],marker="o",color="green",label = "purchases")
plt.scatter(data['CREDIT_LIMIT'][:100],data.PAYMENTS[:100],marker="o",color="r",label="PAYMENTS")
# plt.scatter(data['BALANCE_FREQUENCY'],data.CREDIT_LIMIT,marker="o",color="blue")
plt.xlabel("PAYMENTS")
plt.ylabel("CREDIT_LIMIT")
plt.legend()

In [ ]:
plt.scatter(data['CREDIT_LIMIT'][:100],data.CASH_ADVANCE[:100],marker="o",color="blue",label="Cash in Advance")
plt.xlabel("PURCHASES")
plt.ylabel("CREDIT_LIMIT")
plt.legend()

# FEATURE ENGINEERING 

In [ ]:
# DATA CLEANING 

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna() # droping the empty cells 

In [ ]:
data['CUST_ID'].duplicated().sum() # checking for the duplicated columns in my dataset 

In [ ]:
# outliers in the data set
# BALANCE_FREQUENCY: How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
# check for outliers in the credit card limit 

In [ ]:
frequency_gretaer_1 = data[data['BALANCE_FREQUENCY']>1] # get the values that are greter than 1 in the column
len(frequency_gretaer_1)

In [ ]:
q1 = data['CREDIT_LIMIT'].quantile(0.25)
q3 = data['CREDIT_LIMIT'].quantile(0.75)

IQR = q3-q1
print (f"the q1 is {q1}")
print (f"the q2 is {q3}")
print (f"the IQR is {IQR}")

lower_limit = q1-1.5*(IQR)
upper_limit = q3+1.5 * (IQR)

print (f"the lower limit is {lower_limit}")
print (f"the upper limit  is {upper_limit}")
outliers = data[(data["CREDIT_LIMIT"]<lower_limit) | (data["CREDIT_LIMIT"]>upper_limit)]
print (f"the number of outliers is {len(outliers)}")
filtered_data = data[(data["CREDIT_LIMIT"]>= lower_limit) & (data["CREDIT_LIMIT"] <= upper_limit)]

In [ ]:
filtered_data.shape

In [ ]:
data2 = filtered_data.copy()

In [ ]:
# FETURE SELECTION 
from sklearn.model_selection import train_test_split
# xtrain,xtest,ytain,ytest = train_test_split()
from sklearn.feature_selection import VarianceThreshold 

In [ ]:
def varience(dataset,threshold):
    get_variance = VarianceThreshold(threshold=threshold)
    get_variance.fit(dataset)
    return pd.Series(get_variance.get_support(),index=dataset.columns)

In [ ]:
varience(data2.drop(columns=['CUST_ID']),0.1)

In [ ]:
data2.BALANCE_FREQUENCY[:5] # BALANCE_FREQUENCY: How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)

In [ ]:
# Highly correwlated features in my dataset can imply the same thing especialy when its posituve correlation ()

In [ ]:
import seaborn as sns
plt.title("CORRELATIONS")
sns.heatmap(data2.select_dtypes(exclude='object').corr(),annot=True)

# CLUSTERING ALGORITHMS TO GET THE DIFFERENT CLUSTERS LABELS 

In [ ]:
# NOTE :-  We perform clusters to get the labels (Target)

In [ ]:
data2.columns

In [ ]:
# KMEANS CLUSTERING ALGORITHMS 
from sklearn.cluster import KMeans,AgglomerativeClustering
# Perform partitional clustering using the K-means algorithm. Then, plot the clusters

In [ ]:
clusters = AgglomerativeClustering(n_clusters=2,metric='euclidean')
clusters.fit(data2[['PURCHASES','CREDIT_LIMIT']].iloc[:100])

In [ ]:
labels = clusters.labels_[:100]
np.unique(labels) # you realise that the model only generated 2 unique lables 

In [ ]:
parameters = clusters.get_params()
parameters

In [ ]:
plt.scatter(data2['PURCHASES'].iloc[:100], data2['CREDIT_LIMIT'].iloc[:100], c=labels[:100], cmap='viridis')
# plt.scatter(data2['PAYMENTS'], data2['CREDIT_LIMIT'], c=labels, cmap='viridis')
plt.title('Clusters of Purchases and Credit Limit (inheritance groups)')
plt.xlabel('Purchases')
plt.ylabel('Credit Limit')
# plt.legend()
plt.show()

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
# Perform partitional clustering using the K-means algorithm. Then, plot the clusters

In [ ]:
kmeans_clusters = KMeans(n_clusters=2,random_state=100)
kmeans_clusters.fit(data2[['PURCHASES','CREDIT_LIMIT']].iloc[:100])

In [ ]:
labels = kmeans_clusters.labels_
labels

In [ ]:
SSE = []
k_range = range(1,10)
for k in k_range:
    kmeans = KMeans(n_clusters=k,random_state=100)
    kmeans.fit(data2[['PURCHASES','CREDIT_LIMIT']].iloc[:100])
    SSE.append(kmeans.inertia_)

In [ ]:
# Elbow plot 
plt.plot(k_range,SSE,c='blue',marker='o')
plt.xlabel("K values ")
plt.ylabel("SSE")
plt.title("ELBOW PLOT")
plt.grid()

In [ ]:
plt.figure(figsize=(10, 8))
# Plotting the first 100 points
plt.scatter(data2['PURCHASES'][:100], data2['CREDIT_LIMIT'][:100], c=labels[:100], cmap='viridis', marker='o')
plt.scatter(kmeans_clusters.cluster_centers_[:, 0], kmeans_clusters.cluster_centers_[:, 1], c='red', marker='x')  # Plotting centroids
plt.title('K-Means Clustering of Purchases and Credit Limit (First 100 Points)')
plt.xlabel('Purchases')
plt.ylabel('Credit Limit')
plt.grid()
plt.show()

# Interpret the results

In [ ]:
# from the algorithms above Hirrachial model is better in this case because we dont know the number of clusters that we have
# in this case despite both of the having the same output 
# 2. the model shows that the best labels we have is 0 and 1 we can only establish to clusters from the samples 

In [ ]:
# testing the model algorithm
'''
from the dataset the first input was :
  CUST_ID	    BALANCE_FREQUENCY	PURCHASES	PAYMENTS	CREDIT_LIMIT	CASH_ADVANCE
0    C10001	    0.818182	        95.40	     201.802084	  1000.0	     0.000000 
1	 C10002  	0.909091	        0.00	    4103.032597	  7000.0	     6442.945483
6934 C17123	    1.000000	        0.00	    263.793622	  1200.0	     119.746155
'''

# according to this we want to predict the customer label of this customer will be ? basing on the PURCHASES COLUMN AND CREDIT_LIMIT
print(f"the first {kmeans_clusters.predict([[95,1000]])}") # 1st
print(f"the 2nd client {kmeans_clusters.predict([[95,1000]])}") # second
print(kmeans_clusters.predict([[0,1200]]) )# the customer 6934

In [ ]:
# so the customers we can classify him as cluster 0 